In [1]:
import pandas as pd
import numpy as np

In [2]:
df_summs_raw = pd.read_csv("all_games.csv")
df_sales_raw = pd.read_csv("vgsales.csv")
df_summs = df_summs_raw.copy()
df_sales = df_sales_raw.copy()

df_sales = df_sales.drop(["Rank"],axis=1)
df_summs = df_summs.rename(columns={'name': 'Name'})
df_summs = df_summs.rename(columns={'platform': 'Platform'})

In [3]:
# drop rows with nans in the important columns
print(df_sales.shape)
print(df_summs.shape)

df_sales = df_sales.dropna(axis=0,subset=['Name', 'Year', 'Global_Sales'])
df_summs = df_summs.dropna(axis=0,subset=['Name', 'release_date', 'summary'])

print(df_sales.shape)
print(df_summs.shape)

(16598, 10)
(18800, 6)
(16327, 10)
(18686, 6)


In [4]:
# Make platform names same in each dataframe
print(df_sales["Platform"].unique())
print(df_summs["Platform"].unique())

plats = {"N64":"Nintendo 64", "PS":"PlayStation", "PS3":"PlayStation 3", "DC":"Dreamcast", \
        "X360":"Xbox 360", "XOne":"Xbox One", "PS2":"PlayStation 2", "PS4":"PlayStation 4", \
        "GC":"GameCube", "PSV":"PlayStation Vita", "GBA":"Game Boy Advance", "WiiU":"Wii U", "XB":"Xbox"}
for key in plats.keys():
    df_sales['Platform'] = df_sales['Platform'].replace(key,plats[key])

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' '3DS' 'PS4' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'GC' 'WiiU' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
[' Nintendo 64' ' PlayStation' ' PlayStation 3' ' Dreamcast' ' Xbox 360'
 ' Wii' ' Xbox One' ' PC' ' Switch' ' PlayStation 2' ' PlayStation 4'
 ' GameCube' ' Xbox' ' Wii U' ' Game Boy Advance' ' 3DS' ' Xbox Series X'
 ' DS' ' PlayStation Vita' ' PlayStation 5' ' PSP' ' Stadia']


In [5]:
# strip any leading or trailing spaces in the columns I'm going to merge on
df_sales["Name"] = df_sales["Name"].str.strip()
df_sales["Platform"] = df_sales["Platform"].str.strip()

df_summs["Name"] = df_summs["Name"].str.strip()
df_summs["Platform"] = df_summs["Platform"].str.strip()

In [6]:
# merge the dataframes on game name, year, and platform
df_sales["Year"] = df_sales["Year"].astype(int)
df_summs["Year"] = df_summs["release_date"].str[-4:].astype(int)
df = df_summs.merge(df_sales, how="inner", on=["Name","Year","Platform"])

In [7]:
print(df.shape)
df

(5294, 14)


,Name,Platform,release_date,summary,meta_score,user_review,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4,2000,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7,2008,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4,1999,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9,2008,Action,Take-Two Interactive,6.76,3.10,0.14,1.03,11.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Pulse Racer,Xbox,"January 2, 2003",Pulse Racer takes you to a future where racers...,24,2.2,2003,Racing,Jaleco,0.01,0.00,0.00,0.00,0.02
5290,Fighter Within,Xbox One,"November 19, 2013",Unleash your inner fighter to beat your friend...,23,2.8,2013,Fighting,Ubisoft,0.11,0.07,0.00,0.02,0.19
5291,Homie Rollerz,DS,"March 5, 2008","Homie Rollerz is a fast-paced, mayhem-laden ka...",23,3.0,2008,Racing,Destineer,0.07,0.00,0.00,0.01,0.07
5292,Charlie's Angels,GameCube,"July 9, 2003","Join Natalie, Dylan, and Alex for an intense a...",23,4.3,2003,Action,Ubisoft,0.01,0.00,0.00,0.00,0.02


In [8]:
df.dtypes

Name             object
Platform         object
release_date     object
summary          object
meta_score        int64
user_review      object
Year              int64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object

In [9]:
# try to convert user_review column to float
#for i in df["user_review"]:
#    try:
#        i = float(i)
#    except:
#        print(i)
#        break

In [10]:
# how many rows have user_review = tbd ? Drop them
#print(df[df["user_review"] == "tbd"].shape)
#df = df.drop(df[df["user_review"] == "tbd"].index,axis=0)
#df.shape

In [11]:
# convert user_review to float
#df["user_review"] = df["user_review"].astype(float)
#df.dtypes

In [12]:
df.describe()

,meta_score,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,5294.000000,5294.000000,5294.00000,5294.000000,5294.000000,5294.000000,5294.000000
mean,73.152815,2006.986211,0.44493,0.262877,0.055059,0.089365,0.852439
std,11.880342,4.107427,1.06396,0.734283,0.260976,0.276081,2.110870
min,22.000000,1996.000000,0.00000,0.000000,0.000000,0.000000,0.010000
25%,66.000000,2004.000000,0.07000,0.020000,0.000000,0.010000,0.120000
50%,74.000000,2007.000000,0.17000,0.070000,0.000000,0.020000,0.330000
75%,81.000000,2010.000000,0.45000,0.240000,0.010000,0.080000,0.850000
max,99.000000,2016.000000,41.49000,29.020000,6.500000,10.570000,82.740000


In [13]:
# convert release_date to time
df["release_date"] = pd.to_datetime(df["release_date"])
print(df.dtypes)
df.head()

Name                    object
Platform                object
release_date    datetime64[ns]
summary                 object
meta_score               int64
user_review             object
Year                     int64
Genre                   object
Publisher               object
NA_Sales               float64
EU_Sales               float64
JP_Sales               float64
Other_Sales            float64
Global_Sales           float64
dtype: object


,Name,Platform,release_date,summary,meta_score,user_review,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,As most major publishers' development efforts ...,98,7.4,2000,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7,2008,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,SoulCalibur,Dreamcast,1999-09-08,"This is a tale of souls and swords, transcendi...",98,8.4,1999,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,Grand Theft Auto IV,Xbox 360,2008-04-29,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9,2008,Action,Take-Two Interactive,6.76,3.10,0.14,1.03,11.02


In [14]:
# What summaries are really short? They look fine
df = df.reset_index(drop=True)
for i in range(len(df["summary"])):
    if len(df["summary"][i]) < 40:
        print(df["Name"][i])
        print(df["summary"][i])

Art Academy: Home Studio
Also known as Art Academy: Atelier.


In [15]:
# make summaries lower case
df['summary'] = df['summary'].str.lower()

In [16]:
# remove special characters from summary
chars = ['\[', '\]', ',', ';', '.', '!', '?', ':', '\'s', '\'ll', '\'', '&', '*', ' - ', '(', ')']
for i in chars:
    df['summary'] = df['summary'].str.replace(i,'')
df['summary'] = df['summary'].str.replace('–',' ')
df['summary'] = df['summary'].str.replace('-',' ')

/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_51970/2178734456.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['summary'] = df['summary'].str.replace(i,'')
/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_51970/2178734456.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['summary'] = df['summary'].str.replace(i,'')


In [17]:
# what are the most common words used in all summaries?
from collections import Counter
most_freq = Counter(" ".join(df["summary"]).split()).most_common(100)
print(most_freq)

[('the', 36190), ('and', 22893), ('of', 16741), ('to', 15819), ('a', 12429), ('in', 9974), ('with', 6588), ('as', 5264), ('you', 4978), ('new', 4813), ('your', 4686), ('on', 4276), ('for', 4072), ('players', 3846), ('game', 3815), ('is', 3611), ('from', 3469), ('an', 3159), ('that', 3136), ('their', 2515), ('world', 2514), ('can', 2435), ('all', 2416), ('or', 2266), ('by', 2224), ('more', 1976), ('will', 1922), ('play', 1753), ('up', 1678), ('through', 1639), ('player', 1625), ('his', 1603), ('are', 1544), ('experience', 1525), ('into', 1519), ('action', 1440), ('it', 1405), ('time', 1342), ('this', 1328), ('features', 1291), ('take', 1280), ('mode', 1247), ('battle', 1234), ('they', 1231), ('be', 1226), ('including', 1222), ('has', 1205), ('have', 1175), ('one', 1170), ('gameplay', 1142), ('at', 1093), ('characters', 1083), ('control', 1021), ('also', 977), ('over', 975), ('games', 973), ('out', 929), ('each', 928), ('first', 917), ('most', 899), ('team', 895), ('system', 889), ('agai

In [20]:
# remove fluffer words
words = [' the ', ' and ', ' of ', ' to ', ' a ', ' in ', ' with ', ' your ', ' is ', ' you ', \
         ' as ', ' on ', ' for ', ' from ', ' an ', ' that ', ' by ', ' their ', ' will ', ' or ', \
         ' can ', ' all ', ' are ', ' this ', ' it ', ' into ', ' up ', ' be ', ' has ', \
         ' have ', ' its ', ' them ', ' which ', ' than ', ' but ', ' at ', ' was ', ' what ', ' when ',\
         ' take ', ' even ', ' been ', ' while ', ' who ', ' like ', ' get ', ' where ', ' they ']
for i in words:
    df['summary'] = df['summary'].str.replace(i,' ')

# what are most frequent now?
most_freq = Counter(" ".join(df["summary"]).split()).most_common(100)
print(most_freq)

[('new', 4813), ('players', 3846), ('game', 3815), ('world', 2514), ('more', 1976), ('play', 1753), ('through', 1639), ('player', 1625), ('his', 1603), ('experience', 1525), ('action', 1440), ('time', 1342), ('features', 1291), ('mode', 1247), ('battle', 1234), ('including', 1222), ('one', 1170), ('gameplay', 1142), ('characters', 1083), ('control', 1021), ('also', 977), ('over', 975), ('games', 973), ('out', 929), ('each', 928), ('first', 917), ('most', 899), ('team', 895), ('system', 889), ('against', 885), ('unique', 873), ('online', 853), ('combat', 851), ('own', 845), ('way', 826), ('series', 825), ('use', 814), ('modes', 802), ('multiplayer', 793), ('story', 765), ('now', 754), ('must', 749), ('city', 746), ('2', 728), ('enemies', 721), ('weapons', 713), ('war', 706), ('based', 701), ('other', 699), ('gamers', 689), ('set', 679), ('create', 672), ('adventure', 669), ('he', 668), ('fight', 667), ('power', 657), ('only', 643), ('two', 639), ('racing', 639), ('ever', 618), ('life', 

In [21]:
# save the cleaned df
df.to_pickle("cleaned_df.pkl")